In [24]:
from typing import Literal, TypedDict
from langgraph.types import interrupt, Command
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from pydantic import BaseModel

class State(BaseModel):
    llm_output: str
    question: str
    
def human_approval(state):
    print("Human approval node")
    answers = interrupt(
        {
            "interrupt_type": "questions",
            "questions": ["This is a question", "This is another question"],
            # Surface the output that should be
            # reviewed and approved by the human.
            "llm_output": state.llm_output
        }
    )

    print("Done", answers)

# Add the node to the graph in an appropriate location
# and connect it to the relevant nodes.
graph_builder = StateGraph(State)
graph_builder.set_entry_point("human_approval")
graph_builder.add_node("human_approval", human_approval)
graph_builder.add_edge("human_approval", END)

thread_config = {"configurable": {"thread_id": "1"}}
checkpointer = MemorySaver()
graph = graph_builder.compile(checkpointer=checkpointer)

initial_state = State(llm_output="This is a test", question="Is this correct?")
result = graph.invoke(initial_state, config=thread_config)
# After running the graph and hitting the interrupt, the graph will pause.
# Resume it with either an approval or rejection.
#graph.invoke(Command(resume=True), config=thread_config)


Human approval node


In [25]:
state = graph.get_state(thread_config)
tasks = state.tasks[0].interrupts[0].value
tasks

{'interrupt_type': 'questions',
 'questions': ['This is a question', 'This is another question'],
 'llm_output': 'This is a test'}

In [26]:
answers = {}
if tasks["interrupt_type"] == "questions":
    for question in tasks["questions"]:
        print(f"Question: {question}")
        answer = input("Answer: ")
        answers[question] = answer

Question: This is a question
Question: This is another question


In [27]:
answers

{'This is a question': 'a', 'This is another question': 'b'}

In [28]:
final_state = await graph.ainvoke(Command(resume=answers), config=thread_config)
final_state

Human approval node
Done {'This is a question': 'a', 'This is another question': 'b'}


{'llm_output': 'This is a test', 'question': 'Is this correct?'}

In [29]:
state = graph.get_state(thread_config)
state


StateSnapshot(values={'llm_output': 'This is a test', 'question': 'Is this correct?'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1eff5cf8-96d4-618e-8001-8350e00d2ca4'}}, metadata={'source': 'loop', 'writes': {'human_approval': None}, 'thread_id': '1', 'step': 1, 'parents': {}}, created_at='2025-02-28T12:28:38.444875+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1eff5cf8-7881-606a-8000-0e778d11eafe'}}, tasks=())